In [ ]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta
from itertools import chain

from datetime import date
from shapely.geometry import MultiPolygon


sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [ ]:
def remerge_largefire(fires):
    '''
    If two final feds perimeters intersect spatialy, check if one ended before the other began. If yes, give it the ID of the earlier perimeter. 
    Note: Could optionally be spitting out time differences, or sorting by them. For Quebec, was a max of 22 days, min of 1 day. Theoretically, not sure why the 1 day perimeter wasn't merged by feds. 
    '''
    # Get an id, first/ last t per ID, and a geometry
    first_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.min().reset_index()
    last_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.max().reset_index()
    plot_last = fires.merge(last_perims, on = ["fireID", "t"], how = 'right')
    plot_first = fires.merge(first_perims, on = ["fireID", "t"], how = 'right')
    plot_last = plot_last[["fireID", "t", "geometry"]]
    plot_first = plot_first[["fireID", "t", "geometry"]]
                    
    # Check what perimeters spatially intersect into anouther one through time
    last_in_last = plot_last.sjoin(plot_last, how = "left", predicate = "intersects")
    lil = last_in_last.groupby(["fireID_left",]).t_right.min().reset_index()
    id_with_max_time = lil.merge(last_in_last[["t_right", "t_left", "geometry", "fireID_right", "fireID_left"]], on = ["t_right", "fireID_left"], how = "right")

    id_with_max_time = id_with_max_time.rename(columns={"fireID_right": "mergeID", 
                                     "fireID_left" :"fireID",
                                     "t_right":"mergeID_t",
                                     "t_left":"fireID_end_t"
    })

    id_with_max_time_check = id_with_max_time[id_with_max_time.fireID_end_t < id_with_max_time.mergeID_t]

    ### FireID the earlier perimeter that later perimeters are merged into. "mergeID" describes the merge-ey ## I actually think this is wrong. I think I made it so the "fireID" is what is merged into the "mergeID"
    fireID_with_merge = id_with_max_time_check.groupby(["fireID"]).mergeID.unique().reset_index() 
    #print(fireID_with_merge)

    mergeID_with_fireID  = id_with_max_time_check.groupby(["mergeID"]).fireID.unique().reset_index()
    #print(mergeID_with_fireID)
                    
    # Check when the fireID and mergeID started/stopped. 
    get_merge_start = plot_first[["fireID", "t"]].rename(columns={"fireID":"mergeID", 
                                                           "t":"mergeID_start_t"})
    #print(get_merge_start)
    get_fireID_start =  plot_first[["fireID", "t"]].rename(columns={ "t":"fireID_start_t"})
    #print(get_fireID_start)

    id_map = id_with_max_time_check.merge(get_merge_start, on = ["mergeID"])
    id_map = id_map.merge(get_fireID_start, on = ["fireID"])
    id_map = id_map[["fireID","fireID_start_t",  "fireID_end_t", "mergeID", "mergeID_start_t", "mergeID_t"]]
    id_map["time_diff_fireIDend_mergeIDstart"] = id_map.fireID_end_t.astype('datetime64[ns]') - id_map.mergeID_start_t.astype('datetime64[ns]') ## Negative means that mergeID started after fireID ended
    #print(id_map)
    # Below code is just for isolateing perimeters that ended before the later perimetetr began. This is useful for identifying places where the assumptino to invalidate a fire after five days might be wrong. 
#     # Subset to IDs where one fire "ended" before the next fire began                
#     only_IDs_with_negative_dates = id_map[id_map.time_diff_fireIDend_mergeIDstart.dt.days < 0]
    
#     # Go through an reindex just the IDs that overlap in space but not time                
#     fires["old_id"] = ""
#     fire_ids = only_IDs_with_negative_dates.mergeID.unique() ## Gives the IDs of fires to be merged into anouther fire

#     for i in fire_ids:
#         print("start: ")
#         print(fire_ids)
#         min_t = id_with_max_time_check[id_with_max_time_check.mergeID == i].fireID_end_t.min()
#         print(min_t)
#         sm_id_map = id_with_max_time_check[(id_with_max_time_check.mergeID == i) & (id_with_max_time_check.fireID_end_t == min_t)]
#         print(sm_id_map.fireID.values)
#         if(len(sm_id_map) == 0):
#             print("ID", i, " only merges with self")
#             fires["old_id"][fires.fireID == i] = i
#         else:
#             if(len(sm_id_map) != 1):
#                 print("There are two perimeters that intersect with ID",i, " that started at the same time.")
#                 print(sm_id_map)
#                 break

#             fires["old_id"][fires.fireID == i] = i
#             fires["fireID"][fires.fireID == i] = str(*sm_id_map.fireID.values)
                    
#     #return(fires)
    return(id_map)



In [ ]:
path = "/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/Final_dataset_as_of_20240209.csv" 
fires = fv.prep_fire_files(path)

In [ ]:
fires = fires[fires.InterCloud.isna()]
fires.t = pd.to_datetime(fires.t, format='ISO8601')
new_fires = remerge_largefire(fires)

In [ ]:
## How to merge shapes

#shapely.MultiPolgyon(fires.iloc[100].geometry,  fires.iloc[104].geometry)


test_polygon = MultiPolygon([fires.iloc[100].geometry,  fires.iloc[104].geometry])

test_polygon = MultiPolygon(fires[~fires.geometry.isna()].geometry.all())

In [ ]:
#MultiPolygon(fires[~fires.geometry.isna()].geometry.values)

#MultiPolygon([*fires[~fires.geometry.isna()].geometry.values])
#unary_union(fires[~fires.geometry.isna()].geometry)



In [ ]:
def merge_fires_into(newfire, fires):
    '''
    newfire [DataFrame] output from remerge_largefire. 
    fires [GeoDataFrame] geodataframe of all largefires. 
    '''
    fires = fires[~fires.geometry.isna()]
    print("Removing full timeseries of met data.")
    remake_fires = fires.copy() ## Need to do this? 
    remake_fires["composit_ids"] = None
    remake_fires["rows_edited"] = None

    fires.t = fires.t.astype("datetime64[ns]")
    separator = ', '
    #fires["modified_id"] = fires.fireID
    #newfire = newfire.sort_values(by = ["fireID", "fireID_start_t"])
    #newfire.mergeID_start_t = newfire.mergeID_start_t.astype("datetime64[ns]")
    #newfire.mergeID_t = newfire.mergeID_t.astype("datetime64[ns]")
    for m in newfire.mergeID.unique(): ### For each id that had something merged into it
        fireIDs = newfire[newfire.mergeID == m].fireID.unique() ## Get all the fireID of things that were merged into it
        row_mask = (fires.fireID.isin(fireIDs) | fires.fireID.isin([m]) ) ## Get the IDs that will merge into m and m
        #print(row_mask)
        times = fires[row_mask].t.unique()
        #print(len(times))
       # print(m)
        for t in times:
            #print(t)
            row_mask_f = (row_mask) & (fires.t.astype("str") == str(t))
            row_mask_t = (remake_fires.fireID == m) & (remake_fires.t.astype("str") == str(t))
            if row_mask_t.any():  # Check if any rows match row_mask_t
                #print(f"Updating rows for fireID: {m}, time: {t}")

                remake_fires.loc[row_mask_t, "geometry"] = [unary_union(fires[row_mask_f].geometry)]
                remake_fires.loc[row_mask_t, "farea"] = fires.loc[row_mask_f, "farea"].sum()
                remake_fires.loc[row_mask_t, "fperim"] = fires.loc[row_mask_f, "fperim"].sum()
                remake_fires.loc[row_mask_t, "n_pixels"] = fires.loc[row_mask_f, "n_pixels"].sum()
                remake_fires.loc[row_mask_t, "n_newpixels"] = fires.loc[row_mask_f, "n_newpixels"].sum()
                remake_fires.loc[row_mask_t, "flinelen"] = fires.loc[row_mask_f, "flinelen"].sum()
                remake_fires.loc[row_mask_t, "duration"] = fires.loc[row_mask_f, "duration"].max()
                remake_fires.loc[row_mask_t, "pixden"] = fires.loc[row_mask_f, "n_newpixels"].sum() / fires.loc[row_mask_f, "farea"].sum()
                remake_fires.loc[row_mask_t, "meanFRP"] = (fires.loc[row_mask_f, "meanFRP"] * fires.loc[row_mask_f, "farea"]).sum() / fires.loc[row_mask_f, "farea"].sum()
                remake_fires.loc[row_mask_t, "composit_ids"] = separator.join(fires.loc[row_mask_f, "fireID"].unique())
                remake_fires.loc[row_mask_t, "rows_edited"] = "edited"
            else:
                #print("entering into the else")
                remake_fires.loc[row_mask_t, "rows_edited"] = f"No row of {m} at {t}"
    
     
    # fires_with_other_fires = newfire.mergeID.unique()
    # fires_that_didnt_merge = fires[~(fires.fireID.isin(new_fires.fireID) |fires.fireID.isin(new_fires.mergeID)) ].fireID.unique()
    # remake_fires = remake_fires[(remake_fires.fireID.isin(fires_with_other_fires)) |  fires_that_didnt_merge] ## Drop the IDs that will only be merged into something else
    remake_fires = remake_fires[~remake_fires.fireID.isin(newfire.fireID)] ## Drop the IDs that will only be merged into something else
    
    return(remake_fires)

In [ ]:
fires[(fires.fireID == "10713") & (fires.t.astype("str") == "2023-07-16 00:00:00")]

In [ ]:
test =  merge_fires_into(new_fires, fires)

In [ ]:
test.test_of_edit

In [ ]:
test.test_of_edit.unique()

In [ ]:
print(len(fires.fireID.unique()))
print(len(test.fireID.unique()))


In [ ]:
new_fires[new_fires.fireID.isin(new_fires.mergeID)]

In [ ]:
# large_8495 = fv.load_large_fire("8495", "2023", "BOREAL_NRT_3571_DPS", layer = "nfplist")

In [ ]:
# large_8505 = fv.load_large_fire("8505", "2023", "BOREAL_NRT_3571_DPS", layer = "nfplist")

In [ ]:
# large_8505.t = large_8505.t.astype("str")
# large_8505.datetime = large_8505.datetime.astype("str")

# large_8495.t = large_8495.t.astype("str")
# large_8495.datetime = large_8495.datetime.astype("str")

In [ ]:
# # ### Checking to see if the numbers for the cummulative statistics are wrong (ie, how are the number of pixels added in before the merge? after the merge?)

# fires.t = fires.t.astype("str")

# # 8645
# foi = '8495'

# bbox = fires[(fires.fireID == foi) & (~fires.geometry.isna()) & (fires.t == fires[(fires.fireID == foi) & (~fires.geometry.isna())].t.max())].bounds
# #fires.t = fires.t.astype("datetime64[ns]")
# #fires = fires.sort_values(by = "t", ascending= False)
# #fires.t = fires.t.astype("str")

# #some_ids = fires.loc[(fires.t <= "2023-06-05"), ['fireID', 't', "n_pixels", 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].fireID.unique()
# m = fires.loc[(fires.t <= "2023-06-05"), ['fireID', 't', "n_pixels", "n_newpixels", 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")
# m = large_8495[large_8495.t <= "2023-06-05"].explore(m = m)
# m = large_8505[large_8505.t <= "2023-06-05"].explore(m = m, color = "light blue")
# m

In [ ]:
 ### Checking to see if the numbers for the cummulative statistics are wrong (ie, how are the number of pixels added in before the merge? after the merge?)
fires.t = fires.t.astype("datetime64[ns]")
fires = fires.sort_values(by = "t", ascending = False)
fires.t = fires.t.astype("str")

# 8645
foi = '8495'

bbox = fires[(fires.fireID == foi) & (~fires.geometry.isna()) & (fires.t == fires[(fires.fireID == foi) & (~fires.geometry.isna())].t.max())].bounds
#fires.t = fires.t.astype("datetime64[ns]")
#fires = fires.sort_values(by = "t", ascending= False)
#fires.t = fires.t.astype("str")

#some_ids = fires.loc[(fires.t <= "2023-06-05"), ['fireID', 't', "n_pixels", 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].fireID.unique()
m = fires[['fireID', 't', "n_pixels", "n_newpixels", 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")
#m = large_8495[large_8495.t <= "2023-06-05"].explore(m = m)
#m = large_8505[large_8505.t <= "2023-06-05"].explore(m = m, color = "light blue")
m

In [ ]:
test.t.astype("datetime64[ns]")
test = test.sort_values(by = "t", ascending = False)

test = test[['fireID', 't', 'geometry', 'n_pixels', 'n_newpixels',
       'farea', 'fperim', 'flinelen', 'duration', 'pixden', 'meanFRP',
       'lon_centroid', 'lat_centroid', 'GEOS-5.IMERGEARLY',
       'FWI','composit_ids']]
test.t = test.t.astype("str")

In [ ]:
test.columns

In [ ]:
test[(test.fireID == "8495")].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")

In [ ]:
#print(len(large_8505[large_8505.t <= "2023-06-03 00:00:00"]))
#print(len(large_8495[large_8495.t <= "2023-06-03 00:00:00"])) ## Before merge pixels are counted seperately

In [ ]:
#print(len(large_8495[large_8495.t <= "2023-06-03 12:00:00"])) ### NUmber of pixels post-merge represents the cummulative of the smaller plus the commulative of the bigger fire. 

## Conclusions

What the above code tells me is that up untill the moment of the merge, the perimeters are treated seperately, but one the merge happens they are counted together. 

In [ ]:
sml = 4 + 8 + 7 + 3

big = 1 + 6 +100 + 5 + 9 + 1 + 4 + 8

print(big / 8)

print(sml / 4)

print((big / 8) + (sml / 4))
print((big + sml)/ 12)

In [ ]:
#	8244


foi = '12596'

bbox = fires[(fires.fireID == foi) & (~fires.geometry.isna()) & (fires.t == fires[(fires.fireID == foi) & (~fires.geometry.isna())].t.max())].bounds
fires.t = fires.t.astype("datetime64[ns]")
fires = fires.sort_values(by = "t", ascending= False)
fires.t = fires.t.astype("str")

some_ids = fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].fireID.unique()
fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")